# ETL: extraction, tranformation and loading

The main purpose of this notebook is the extraction of the data from the csv files of the MIMIC-III dataset and the insertion on the mimic3 database in postgres using the model already created and the library `sequel`.

At first we tried to have a general function to import any csv file, but then we discovered the problem of type casting (i.e. cast to DateTime). So we wrote a script for each table.

The complete MIMIC-III database is far too big for our simple analysis, so we decided to use a reduced version. The main tables have been loaded completely (such as Patients, Admissions, Icustay...), whereas the other were trimmed at the first 10000 lines of the csv file, in order to have a realistic database keeping it lighweight.

## Imports

These files contains the connection to the database and the classes of the model.

In [2]:
load 'connection.rb'
load 'model.rb'
require 'csv'
nil

### Constants
- **PATH**: The path to the folder containing all the csv files

In [3]:
PATH = "/mimic3/1.4/"
nil

### Useful function
- **parse_date**: Parse a date; if it's `nil` return `nil`
- **parse_int**: Parse a int; if it's `nil` return `nil`
- **parse_double**: Parse a double; if it's `nil` return `nil`

In [4]:
def parse_date(date)
  return date.nil? ? nil : DateTime.parse(date)
end

def parse_int(n)
  return n.nil? ? nil : n.to_i
end

def parse_double(n)
  return n.nil? ? nil : n.to_f
end
nil

## Patients

In [33]:
count = 0

CSV.foreach(PATH + "PATIENTS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  row_id = row['row_id'].to_i
  subject_id = row['subject_id'].to_i
  gender = row['gender']
  dob = DateTime.parse(row['dob'])
  dod = row['dod'].nil? ? nil : DateTime.parse(row['dod'])
  dod_hosp = row['dod_hosp'].nil? ? nil : DateTime.parse(row['dod_hosp'])
  dod_ssn = row['dod_ssn'].nil? ? nil : DateTime.parse(row['dod_ssn'])
  expire_flag = row['expire_flag'].to_i
  
  Patient.create(
    row_id: row_id,
    subject_id: subject_id,
    gender: gender,
    dob: dob,
    dod: dod,
    dod_hosp: dod_hosp,
    dod_ssn: dod_ssn,
    expire_flag: expire_flag
  )
  
  count += 1
end

puts "Created #{count} patients"

Created 46520 patients


## Admissions

In [35]:
count = 0

CSV.foreach(PATH + "ADMISSIONS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    admittime: DateTime.parse(row['admittime']),
    dischtime: row['dischtime'].nil? ? nil : DateTime.parse(row['dischtime']),
    deathtime: row['deathtime'].nil? ? nil : DateTime.parse(row['deathtime']),
    edregtime: row['edregtime'].nil? ? nil : DateTime.parse(row['edregtime']),
    edouttime: row['edouttime'].nil? ? nil : DateTime.parse(row['edouttime']),
    admission_type: row['admission_type'],
    admission_location: row['admission_location'],
    discharge_location: row['discharge_location'],
    insurance: row['insurance'],
    language: row['language'],
    religion: row['religion'],
    marital_status: row['marital_status'],
    ethnicity: row['ethnicity'],
    diagnosis: row['diagnosis'],
    hospital_expire_flag: row['hospital_expire_flag'].to_i,
    has_chartevents_data: row['has_chartevents_data'].to_i
  }
  
  Admission.create(entry)
  
  count += 1
end

puts "Created #{count} admissions"

Created 58976 admissions


## Drg codes

In [37]:
count = 0

CSV.foreach(PATH + "DRGCODES_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    drg_type: row['drg_type'],
    drg_code: row['drg_code'],
    description: row['description'],
    drg_severity: row['drg_severity'].to_i,
    drg_mortality: row['drg_mortality'].to_i
  }
  
  DrgCode.create(entry)
  
  count += 1
end

puts "Created #{count} DRGs"

Created 9999 DRGs


## Microbiology events

In [40]:
count = 0

CSV.foreach(PATH + "MICROBIOLOGYEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    chartdate: row['chartdate'].nil? ? nil : DateTime.parse(row['chartdate']),
    charttime: row['charttime'].nil? ? nil : DateTime.parse(row['charttime']),
    spec_itemid: row['spec_itemid'].nil? ? nil : row['chartdate'].to_i,
    spec_type_desc: row['spec_type_desc'],
    org_itemid: row['org_itemid'].nil? ? nil : row['org_itemid'].to_i,
    org_name: row['org_name'],
    isolate_num: row['isolate_num'].nil? ? nil : row['isolate_num'].to_i,
    ab_itemid: row['ab_itemid'].nil? ? nil : row['ab_itemid'].to_i,
    ab_name: row['ab_name'],
    dilution_text: row['dilution_text'],
    dilution_comparison: row['dilution_comparison'],
    dilution_value: row['dilution_value'].nil? ? nil : row['dilution_value'].to_f,
    interpretation: row['interpretation']
  }
  
  MicrobiologyEvent.create(entry)
  
  count += 1
end

puts "Created #{count} Microbiology events"

Created 9999 Microbiology events


## Callouts

In [41]:
count = 0

CSV.foreach(PATH + "CALLOUT.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    submit_wardid: row['submit_wardid'].nil? ? nil : row['submit_wardid'].to_i,
    submit_careunit: row['submit_careunit'],
    curr_wardid: row['curr_wardid'].nil? ? nil : row['curr_wardid'].to_i,
    curr_careunit: row['curr_careunit'],
    callout_wardid: row['callout_wardid'].nil? ? nil : row['callout_wardid'].to_i,
    callout_service: row['callout_service'],
    request_tele: row['request_tele'].nil? ? nil : row['request_tele'].to_i,
    request_resp: row['request_resp'].nil? ? nil : row['request_resp'].to_i,
    request_cdiff: row['request_cdiff'].nil? ? nil : row['request_cdiff'].to_i,
    request_mrsa: row['request_mrsa'].nil? ? nil : row['request_mrsa'].to_i,
    request_vre: row['request_vre'].nil? ? nil : row['request_vre'].to_i,
    callout_status: row['callout_status'],
    callout_outcome: row['callout_outcome'],
    discharge_wardid: row['discharge_wardid'].nil? ? nil : row['discharge_wardid'].to_i,
    acknowledge_status: row['acknowledge_status'],
    createtime: row['createtime'].nil? ? nil : DateTime.parse(row['createtime']),
    updatetime: row['updatetime'].nil? ? nil : DateTime.parse(row['updatetime']),
    acknowledgetime: row['acknowledgetime'].nil? ? nil : DateTime.parse(row['acknowledgetime']),
    outcometime: row['outcometime'].nil? ? nil : DateTime.parse(row['outcometime']),
    firstreservationtime: row['firstreservationtime'].nil? ? nil : DateTime.parse(row['firstreservationtime']),
    currentreservationtime: row['currentreservationtime'].nil? ? nil : DateTime.parse(row['currentreservationtime']),
  }
  
  Callout.create(entry)
  
  count += 1
end

puts "Created #{count} Callouts"

Created 34499 Callouts


## Diagnoses ICD

In [43]:
count = 0

CSV.foreach(PATH + "DIAGNOSES_ICD_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: parse_int(row['row_id']),
    subject_id: parse_int(row['subject_id']),
    hadm_id: parse_int(row['hadm_id']),    
    seq_num: parse_int(row['seq_num']),
    icd9_code: row['icd9_code'],
  }
  
  DiagnoseIcd.create(entry)
  
  count += 1
end

puts "Created #{count} Diagnoses Icd"

Created 9999 Diagnoses Icd


At this point we realized that it could be useful to have functions to parse integers and dates returning nil if the string was nil. So we implemented them in the top of the notebook and used them from here on

## Cpt Events

In [44]:
count = 0

CSV.foreach(PATH + "CPTEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,    
    costcenter: row['costcenter'],
    chartdate: parse_date(row['chartdate']),
    cpt_cd: row['cpt_cd'],
    cpt_number: parse_int(row['cpt_number']),
    cpt_suffix: row['cpt_suffix'],
    ticket_id_seq: parse_int(row['ticket_id_seq']),
    sectionheader: row['sectionheader'],
    subsectionheader: row['subsectionheader'],
    description: row['description']
  }
  
  CptEvent.create(entry)
  
  count += 1
end

puts "Created #{count} Cpt events"

Created 9999 Cpt events


## Services

In [46]:
count = 0

CSV.foreach(PATH + "SERVICES_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    transfertime: parse_date(row['transfertime']),
    prev_service: row['subsectionheader'],
    curr_service: row['description']
  }
  
  Service.create(entry)
  
  count += 1
end

puts "Created #{count} services"

Created 9999 services


## Procedures Icd

In [47]:
count = 0

CSV.foreach(PATH + "PROCEDURES_ICD_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    seq_num: parse_int(row['seq_num']),
    icd9_code: row['icd9_code']
  }
  
  ProcedureIcd.create(entry)
  
  count += 1
end

puts "Created #{count} Procedures Icd"

Created 9999 Procedures Icd


## Icustays

In [48]:
count = 0

CSV.foreach(PATH + "ICUSTAYS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    dbsource: row['dbsource'],
    first_careunit: row['first_careunit'],
    last_careunit: row['last_careunit'],
    first_wardid: parse_int(row['first_wardid']),
    last_wardid: parse_int(row['last_wardid']),
    intime: parse_date(row['intime']),
    outtime: parse_date(row['outtime']),
    los: parse_double(row['los'])
  }
  
  Icustay.create(entry)
  
  count += 1
end

puts "Created #{count} icustays"

Created 61532 icustays


## Prescriptions

In [52]:
count = 0

CSV.foreach(PATH + "PRESCRIPTIONS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    startdate: parse_date(row['startdate']),
    enddate: parse_date(row['enddate']),
    drug_type: row['drug_type'],
    drug: row['drug'],
    drug_name_poe: row['drug_name_poe'],
    drug_name_generic: row['drug_name_generic'],
    formulary_drug_cd: row['formulary_drug_cd'],
    gsn: row['gsn'],
    ndc: row['ndc'],
    prod_strength: row['prod_strength'],
    dose_val_rx: row['dose_val_rx'],
    dose_unit_rx: row['dose_unit_rx'],
    form_val_disp: row['form_val_disp'],
    form_unit_disp: row['form_unit_disp'],
    route: row['route'],
  }
  
  Prescription.create(entry)
  
  count += 1
end

puts "Created #{count} prescriptions"

Created 9999 prescriptions


## Transfers

In [54]:
count = 0

CSV.foreach(PATH + "TRANSFERS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    dbsource: row['dbsource'],
    eventtype: row['eventtype'],
    prev_careunit: row['prev_careunit'],
    curr_careunit: row['curr_careunit'],
    prev_wardid: parse_int(row['prev_wardid']),
    curr_wardid: parse_int(row['curr_wardid']),
    intime: parse_date(row['intime']),
    outtime: parse_date(row['outtime']),
    los: parse_int(row['los'])
  }
  
  Transfer.create(entry)
  
  count += 1
end

puts "Created #{count} transfers"

Created 9999 transfers


## Caregivers

In [56]:
count = 0

CSV.foreach(PATH + "CAREGIVERS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    cgid: row['cgid'].to_i,
    label: row['label'],
    description: row['description'],
  }
  
  Caregiver.create(entry)
  
  count += 1
end

puts "Created #{count} caregivers"

Created 7567 caregivers


## Input events CV

In [57]:
count = 0

CSV.foreach(PATH + "INPUTEVENTS_CV_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    charttime: parse_date(row['charttime']),
    itemid: parse_int(row['itemid']),
    amount: parse_double(row['amount']),
    amountuom: row['amountuom'],
    rate: parse_double(row['rate']),
    rateuom: row['rateuom'],
    storetime: parse_date(row['storetime']),
    orderid: parse_int(row['orderid']),
    linkorderid: parse_int(row['linkorderid']),
    stopped: row['stopped'],
    newbottle: parse_int(row['newbottle']),
    originalamount: parse_double(row['originalamount']),
    originalamountuom: row['originalamountuom'],
    originalrate: parse_double(row['originalrate']),
    originalroute: row['originalroute'],
    originalrateuom: row['originalrateuom'],
    originalsite: row['originalsite'],
  }
  
  InputEventCV.create(entry)
  
  count += 1
end

puts "Created #{count} input events cv"

Created 9999 input events cv


## Output events

In [58]:
count = 0

CSV.foreach(PATH + "OUTPUTEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    charttime: parse_date(row['charttime']),
    itemid: parse_int(row['itemid']),
    value: parse_double(row['value']),
    valueuom: row['valueuom'],
    storetime: parse_date(row['storetime']),
    stopped: row['stopped'],
    newbottle: parse_int(row['newbottle']),
    iserror: parse_int(row['iserror'])
  }
  
  OutputEvent.create(entry)
  
  count += 1
end

puts "Created #{count} output events"

Created 9999 output events


## Procedure events MV

In [59]:
count = 0

CSV.foreach(PATH + "PROCEDUREEVENTS_MV_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    starttime: parse_date(row['starttime']),
    endtime: parse_date(row['endtime']),
    itemid: parse_int(row['itemid']),
    value: parse_double(row['value']),
    valueuom: row['valueuom'],
    location: row['location'],
    locationcategory: row['locationcategory'],
    storetime: parse_date(row['storetime']),
    orderid: parse_int(row['orderid']),
    linkorderid: parse_int(row['linkorderid']),
    ordercategoryname: row['ordercategoryname'],
    secondaryordercategoryname: row['secondaryordercategoryname'],
    ordercategorydescription: row['ordercategorydescription'],
    isopenbag: parse_int(row['isopenbag']),
    continueinnextdept: parse_int(row['continueinnextdept']),
    cancelreason: parse_int(row['cancelreason']),
    statusdescription: row['statusdescription'],
    comments_editedby: row['comments_editedby'],
    comments_canceledby: row['comments_canceledby'],
    comments_date: parse_date(row['comments_date'])
  }

  ProcedureEventMV.create(entry)

  count += 1
end

puts "Created #{count} procedure events MV"

Created 9999 procedure events MV


## Input events MV

**NOTE**: There is a discrepancy between <a href="https://mimic.physionet.org/mimictables/inputevents_mv/">Official documentation</a> and the tables generated by the scripts and the headers of the CSV files.

In [60]:
count = 0

CSV.foreach(PATH + "INPUTEVENTS_MV_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    starttime: parse_date(row['starttime']),
    endtime: parse_date(row['endtime']),
    itemid: parse_int(row['itemid']),
    amount: parse_double(row['amount']),
    amountuom: row['amountuom'],
    rate: parse_double(row['rate']),
    rateuom: row['rateuom'],
    storetime: parse_date(row['storetime']),
    orderid: parse_int(row['orderid']),
    linkorderid: parse_int(row['linkorderid']),
    ordercategoryname: row['ordercategoryname'],
    secondaryordercategoryname: row['secondaryordercategoryname'],
    ordercomponenttypedescription: row['ordercomponenttypedescription'],
    ordercategorydescription: row['ordercategorydescription'],
    patientweight: parse_double(row['patientweight']),
    totalamount: parse_double(row['totalamount']),
    totalamountuom: row['totalamountuom'],
    isopenbag: parse_int(row['isopenbag']),
    continueinnextdept: parse_int(row['continueinnextdept']),
    cancelreason: parse_int(row['cancelreason']),
    statusdescription: row['statusdescription'],
    
    # Here start the discrepancy
    # comments_status: row['comments_status'],
    # comments_title: row['comments_title'],
    comments_editedby: row['comments_editedby'],
    comments_canceledby: row['comments_canceledby'],
    
    comments_date: parse_date(row['comments_date']),
    originalamount: parse_double(row['originalamount']),
    originalrate: parse_double(row['originalrate']),
  }
  
  InputEventMV.create(entry)
  
  count += 1
end

puts "Created #{count} input events MV"

Created 9999 input events MV


## Note events

In [63]:
count = 0

CSV.foreach(PATH + "NOTEEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    cgid: row['cgid'].to_i,
    chartdate: parse_date(row['chartdate']),
    charttime: parse_date(row['charttime']),
    storetime: parse_date(row['storetime']),
    category: row['category'],
    description: row['description'],
    iserror: row['iserror'],
    text: row['text']
  }

  NoteEvent.create(entry)

  count += 1
end

puts "Created #{count} note events"

Created 40 note events


## Item definitons

In [64]:
count = 0

CSV.foreach(PATH + "D_ITEMS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    itemid: row['itemid'].to_i,
    label: row['label'],
    abbreviation: row['abbreviation'],
    dbsource: row['dbsource'],
    linksto: row['linksto'],
    category: row['category'],
    unitname: row['unitname'],
    param_type: row['param_type'],
    conceptid: parse_int(row['conceptid'])
  }

  DItem.create(entry)

  count += 1
end

puts "Created #{count} item definitions"

Created 12487 item definitions


## Chart events

In [65]:
count = 0

CSV.foreach(PATH + "CHARTEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    itemid: row['itemid'].to_i,
    charttime: parse_date(row['charttime']),
    storetime: parse_date(row['storetime']),
    value: row['value'],
    valuenum: parse_int(row['value']),
    valueuom: row['valueuom'],
    warning: parse_int(row['warning']),
    error: parse_int(row['error']),
    resultstatus: row['resultstatus'],
    stopped: row['stopped']
  }

  ChartEvent.create(entry)

  count += 1
end

puts "Created #{count} chart events"

Created 9999 chart events


## Datetime events

In [66]:
count = 0

CSV.foreach(PATH + "DATETIMEEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    icustay_id: row['icustay_id'].to_i,
    cgid: row['cgid'].to_i,
    itemid: row['itemid'].to_i,
    charttime: parse_date(row['charttime']),
    storetime: parse_date(row['storetime']),
    value: row['value'],
    valueuom: row['valueuom'],
    warning: parse_int(row['warning']),
    error: parse_int(row['error']),
    resultstatus: row['resultstatus'],
    stopped: row['stopped']
  }

  DateTimeEvent.create(entry)

  count += 1
end

puts "Created #{count} datetime events"

Created 9999 datetime events


## Lab item definition

In [67]:
count = 0

CSV.foreach(PATH + "D_LABITEMS.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    itemid: row['itemid'].to_i,
    label: row['label'],
    fluid: row['fluid'],
    category: row['category'],
    loinc_code: row['loinc_code']
  }

  DLabItem.create(entry)

  count += 1
end

puts "Created #{count} lab item definitions"

Created 753 lab item definitions


## Lab events

In [68]:
count = 0

CSV.foreach(PATH + "LABEVENTS_REDUCED.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    subject_id: row['subject_id'].to_i,
    hadm_id: row['hadm_id'].to_i,
    itemid: row['itemid'].to_i,
    charttime: parse_date(row['charttime']),
    value: row['value'],
    valuenum: parse_double(row['valuenum']),
    valueuom: row['valueuom'],
    flag: row['flag']
  }

  LabEvent.create(entry)

  count += 1
end

puts "Created #{count} lab events"

Created 9999 lab events


## Icd diagnose definitions

In [69]:
count = 0

CSV.foreach(PATH + "D_ICD_DIAGNOSES.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    icd9_code: row['icd9_code'],
    short_title: row['short_title'],
    long_title: row['long_title']
  }

  DIcdDiagnose.create(entry)

  count += 1
end

puts "Created #{count} ICD diagnose definitions"

Created 14567 ICD diagnose definitions


## Cpt definitions

In [70]:
count = 0

CSV.foreach(PATH + "D_CPT.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    category: parse_int(row['category']),
    sectionrange: row['sectionrange'],
    sectionheader: row['sectionheader'],
    subsectionrange: row['subsectionrange'],
    subsectionheader: row['subsectionheader'],
    codesuffix: row['codesuffix'],
    mincodeinsubsection: parse_int(row['mincodeinsubsection']),
    maxcodeinsubsection: parse_int(row['maxcodeinsubsection'])
  }

  DCpt.create(entry)

  count += 1
end

puts "Created #{count} cpt definitions"

Created 134 cpt definitions


## Icd procedure definitions

In [71]:
count = 0

CSV.foreach(PATH + "D_ICD_PROCEDURES.csv", :headers => true, :header_converters => lambda {|h|h.downcase}) do |row|
  entry = {
    row_id: row['row_id'].to_i,
    icd9_code: row['icd9_code'],
    short_title: row['short_title'],
    long_title: row['long_title']
  }

  DIcdProcedure.create(entry)

  count += 1
end

puts "Created #{count} ICD procedure definitions"

Created 3882 ICD procedure definitions


# Test

In [ ]:
Patient.all[0..10].each do |patient|
    puts "ID: #{patient.subject_id},\tCallout number: #{patient.callout.size}"
end
nil

In [ ]:
puts Patient.all.select{|pat| pat.subject_id == 10017}

In [ ]:
puts Admission.first.patient.callout